In [11]:
from google.auth.transport import requests
from googleapiclient import discovery
from google.oauth2 import service_account
from google.cloud import storage
from googleapiclient.http import MediaIoBaseDownload
import io
import threading
from time import sleep
import os
import pandas as pd

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive, GoogleDriveFile
from oauth2client.service_account import ServiceAccountCredentials

from datetime import time

drivers_check_list = []
drivers_checked_list = []
ls_threads = []
ls_tmp = []
ls_error = []
ls_error_files = []
ls_files_to_download = []
count_files_checked = 0
count_files_down = 0
count_files_error = 0
file_down_parms = []
status = None

drivers = [
            ['Ci', '0AHj-PRcbOxSWUk9PVA'],
           ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
           ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
           ['Архив', '0AOP_oKFy4oGJUk9PVA'],
           ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
           ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
           ['Контент', '0ANf3LpamTrsTUk9PVA'],
           ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
           ['Объединенная пивоварня', '0AELFNnmNadJqUk9PVA'],
           ['ОСК Шереметьево', '0ALRUo2DlF1YmUk9PVA'],
           ['Мой Диск _', '0AHpbUawYyiEZUk9PVA'],
    ['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объекты в работе', '0AOxaPQRooOk2Uk9PVA'],
 ['Остров мечты ППР', '0AImPD4jQdMXJUk9PVA'],
 ['Пекарня', '0AIWIhafOWgmHUk9PVA'],
 ['ПОДРЯДЧИКИ', '0ALnZgMLQFfrtUk9PVA'],
 ['СТО ГБО', '0AF2NtBbx_XvYUk9PVA'],
 ['ТЕХВАГОНМАШ', '0AAAyvFQMbstZUk9PVA'],
 ['Транспроект', '0ADKHIbPu575TUk9PVA'],
 ['ФосАгро', '0AD-4c3pLaBV5Uk9PVA'],
 ['ФРАКДЖЕТ-ТУЛЗ', '0ACjAWTbTqvESUk9PVA'],
 ['Холодный склад', '0ALT2Y1wkG7vCUk9PVA'],
 ['Чайка Лаб', '0APbGeMNQtN3MUk9PVA'],
 ['Читинские ключи', '0AA9gFHffXEitUk9PVA'],
 ['Юрист UA', '0AGg7JoJwa_8nUk9PVA']
           ]

storage.blob._DEFAULT_CHUNKSIZE = 10 * 1024* 1024  # 5 MB
storage.blob._MAX_MULTIPART_SIZE = 5 * 1024* 1024  # 5 MB


def transfer_drive(drive_name_id):
    global count_files_down, count_files_error, count_files_checked
    while True:
        driv = GDrive(drive_name_id[0], drive_name_id[1])
        done = driv.download_files()
        if done:
            break


def transfer_drive_thread():
    while True:
        try:
            # service, bucket = get_client()
            # ls_drives = get_list_of_drives(service)
            s_drives = 'TT '

            for drive in drivers:
                if drive[1] not in drivers_check_list and drive[1] not in drivers_checked_list:
                    drivers_check_list.append(drive[1])
                    t = threading.Thread(target=transfer_drive, args=(drive,))
                    t.start()
                    ls_threads.append(t)
                    s_drives += drive[0] + ', '
        except Exception as e:
            ls_error.append(['get_list_of_drives', e])
        ls_tmp.append(f'Завершено------------------------{s_drives}')

        sleep(60*30)



def transfer_drives_thread():
    t = threading.Thread(target=transfer_drive_thread)
    ls_threads.append(t)
    t.start()

    return t


def get_client():
    # Google Drive
    gauth = GoogleAuth()
    scope = ["https://www.googleapis.com/auth/drive"]
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('savvy-concord-374520-bbbb0e3c73c3.json', scope)
    drive = GoogleDrive(gauth)

    credentials_drive = service_account.Credentials.from_service_account_file(
        filename='savvy-concord-374520-bbbb0e3c73c3.json',
        scopes=['https://www.googleapis.com/auth/drive'],
    )
    session = requests.AuthorizedSession(credentials_drive)
    service = discovery.build('drive', 'v3', credentials=credentials_drive)

    #Google Storage
    credentials_storage = service_account.Credentials.from_service_account_file(
        filename='credentials.json',
        scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
    )
    client = storage.Client(credentials=credentials_storage)
    bucket = client.get_bucket('cabinet_bucket')

    return service, drive, bucket


# Общие диски
def get_list_of_drives(service):
    response = service.drives().list(pageSize=100).execute()
    ids = []
    for drive in response['drives']:
        ids.append([drive['name'], drive['id']])
    return ids


class GDrive:
    def __init__(self,  drive_name, drive_id, get_files_folders=True, get_files_fullname=True):
        self.drive_name = drive_name
        self.drive_id = drive_id

        self.drive = None
        self.service = None
        self.bucket = None

        self.get_client()

        self.bucet_main_folder = 'GoogleDriveTransfer'

        self.dict_files = {}
        self.dict_files_fullname = {}
        self.dict_folders = {}

        if get_files_folders:
            self.get_files_folders()
            if get_files_fullname:
                self.get_files_fullname()

        blobs = self.bucket.list_blobs(prefix=f'GoogleDriveTransfer/{drive_name}')
        self.files_blobs = [blob.name for blob in blobs]

    def get_client(self):
        # Google Drive
        gauth = GoogleAuth()
        scope = ["https://www.googleapis.com/auth/drive"]
        gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('savvy-concord-374520-bbbb0e3c73c3.json', scope)
        self.drive = GoogleDrive(gauth)

        credentials_drive = service_account.Credentials.from_service_account_file(
            filename='savvy-concord-374520-bbbb0e3c73c3.json',
            scopes=['https://www.googleapis.com/auth/drive'],
        )
        session = requests.AuthorizedSession(credentials_drive)
        self.service = discovery.build('drive', 'v3', credentials=credentials_drive)

        #Google Storage
        credentials_storage = service_account.Credentials.from_service_account_file(
            filename='credentials.json',
            scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
        )
        client = storage.Client(credentials=credentials_storage)
        self.bucket = client.get_bucket('cabinet_bucket')


    def get_files_folders(self):
        self.get_files()
        self.get_folders()

    def get_folders(self):
        self.dict_folders = {}
        ls_folders = self.drive.ListFile(
            {
                'q': "mimeType='application/vnd.google-apps.folder'",
                'corpora': 'drive',
                'driveId': self.drive_id,
                'includeItemsFromAllDrives': True,
                'supportsAllDrives': True,
            }
        ).GetList()
        for folder in ls_folders:
            self.dict_folders[folder['id']] = folder

    def get_files(self):
        self.dict_files = {}
        ls_files = self.drive.ListFile(
            {
                'q': "mimeType!='application/vnd.google-apps.folder'",
                'corpora': 'drive',
                'driveId': self.drive_id,
                'includeItemsFromAllDrives': True,
                'supportsAllDrives': True,
            }
        ).GetList()
        for file in ls_files:
            self.dict_files[file['id']] = file

    def get_files_fullname(self):
        self.dict_files_fullname = {}
        for file in self.dict_files.values():
            parent_id = file['parents'][0]['id']
            name_full = self.get_full_path(file['title'], parent_id)
            self.dict_files_fullname[file['id']] = name_full

    def get_full_path(self, file_name, parent_id):
        tree = [file_name]
        folder_id = parent_id

        while folder_id != self.drive_id:
            folder = self.dict_folders[folder_id]
            tree.append(folder['title'])
            folder_id = folder['parents'][0]['id']

        tree.append(self.drive_name)
        tree.append(self.bucet_main_folder)

        tree.reverse()
        return '/'.join(tree)

    def download_files(self):
        global count_files_down, count_files_error, count_files_checked
        is_download_files = True

        try:
            for file_id in self.dict_files.keys():
                count_files_checked += 1
                done = self.download_file(file_id)
                if done:
                    is_download_files = False

            ls_tmp.append(f'{self.drive_name}  - Завершено  - {is_download_files}')

            if is_download_files:
                drivers_checked_list.append(self.drive_id)

        except Exception as e:
            ls_error.append([self.drive_name, 'Ошибка', e])
            is_download_files = False

        return is_download_files

    def download_file(self, file_id):
        par = self.get_params_down(file_id)

        try:
            if par['fullname'] not in self.files_blobs:
                if not par['is_propusk']:
                    gdfile = self.dict_files[file_id]
                    size = int(gdfile['fileSize']) / 1024 / 1024
                    if size < 7000:
                        file_down_parms.append([size, par['fullname'], gdfile])
                        if par['is_google_apps']:
                            self.download_file_google_apps(file_id, par)
                        else:
                            self.download_file_no_google_apps(file_id, par)
                        ls_files_to_download.append([size, par['fullname'], gdfile])
                        return True
        except Exception as e:
            ls_error_files.append([size, par['fullname'], gdfile, e])
            return False
        return False

    def download_file_no_google_apps(self, file_id, par):
        global status
        request = self.service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        blob = self.bucket.blob(par['fullname'])
        blob.upload_from_string(fh.getvalue(), timeout=300)
        fh.close()

    def download_file_google_apps(self, file_id, par):
        gdfile = self.dict_files[file_id]

        gdfile.GetContentFile(par['namefile'], mimetype=par['mimeType'])

        blob = self.bucket.blob(par['fullname'])
        blob.upload_from_filename(par['namefile'])

        os.remove(par['namefile'])

    def get_params_down(self, file_id):
        gdfile = self.dict_files[file_id]
        mimeType = gdfile['mimeType']
        namefile = gdfile['title']
        fullname = self.dict_files_fullname[file_id]

        is_google_apps = False
        is_propusk = False

        ls_mime_google_apps = ['application/vnd.google-apps.document', 'application/vnd.google-apps.spreadsheet',
                               'application/vnd.google-apps.presentation', 'application/vnd.google-apps.site',
                               'application/vnd.google-apps.shortcut']

        if mimeType in ls_mime_google_apps:
            is_google_apps = True
            if mimeType == 'application/vnd.google-apps.document':
                mimeType = 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'
                namefile = namefile + '.docx'
                fullname = fullname + '.docx'
            elif mimeType == 'application/vnd.google-apps.spreadsheet':
                mimeType = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                namefile = namefile + '.xlsx'
                fullname = fullname + '.xlsx'
            else:
                is_propusk = True



        res = {
            'namefile': namefile,
            'fullname': fullname,
            'mimeType': mimeType,
            'is_google_apps': is_google_apps,
            'is_propusk': is_propusk,
        }

        return res








In [2]:
t = transfer_drives_thread()

In [3]:
status.progress(), status.resumable_progress / 1024 / 1024, status.total_size / 1024 / 1024

(1.0, 213.78704929351807, 213.78704929351807)

In [4]:
import numpy as np

In [13]:
sum([i[0] for i in ls_files_to_download])

160020.3207464218

In [12]:
[(i[0],i[1]) for i in ls_files_to_download[-10:]]

[(0.08063697814941406,
  'GoogleDriveTransfer/Архив/Пром/Пищевая/РусТомат/Рабочая документация/Разработка/240417-КМ/240417-КМ.1/РАСЧЕТ/Файлы расчета_примежуточные/ПК_Rev17_Назначение груп.SPR'),
 (0.06858158111572266,
  'GoogleDriveTransfer/Архив/Пром/Пищевая/РусТомат/Рабочая документация/Разработка/240417-КМ/240417-КМ.1/РАСЧЕТ/Файлы расчета_примежуточные/ПК_Rev16_Комбинации загружений.SPR'),
 (0.06699275970458984,
  'GoogleDriveTransfer/Архив/Пром/Пищевая/РусТомат/Рабочая документация/Разработка/240417-КМ/240417-КМ.1/РАСЧЕТ/Файлы расчета_примежуточные/ПК_Rev14_Снег.SPR'),
 (2.832662582397461,
  'GoogleDriveTransfer/Архив/Пром/Пищевая/РусТомат/Рабочая документация/Архив/КМ/240417-КМ.1_Производственный корпус.pdf'),
 (0.30265045166015625,
  'GoogleDriveTransfer/Архив/Пром/Пищевая/РусТомат/Проект/Разработка/1412_16-КР/DWG старое/Л-5 !22,02,17м БУФЕТ с сизм по 1-му этажу-Лист - 11 - Буфет со встроенным КПП- План 1-го и 2-го этажей- Разрез 1-1- Фасад 2-1.dwg'),
 (0.8220052719116211,
  'Goo

In [3]:
5<7<9

True

In [4]:
drive_name_id = drivers[-1]
drive_name_id

['Юрист UA', '0AGg7JoJwa_8nUk9PVA']

In [5]:
driv = GDrive(drive_name_id[0], drive_name_id[1])


In [6]:
len(driv.dict_files)

12

In [7]:
len(driv.files_blobs)

12

In [12]:
ls = []
for d in drivers:
    driv = GDrive(d[0], d[1])
    for i in driv.dict_files_fullname.values():
        if i not in driv.files_blobs:
            ls.append(i)

In [2]:
def ff1():
    d = {
        '<1000': 0,
        '1000-2000': 0,
        '2000-3000': 0,
        '3000-4000': 0,
        '4000-5000': 0,
        '5000-6000': 0,
        '6000-7000': 0,
        '7000-8000': 0,
        '8000-9000': 0,
        '9000-10000': 0,
        '>10000': 0,
    }
    ls2 = []
    for i in driv.dict_files.values():
        fileSize = int(i.get('fileSize', 0))
        size = fileSize / 1024 / 1024
        if size < 1000:
            d['<1000'] += 1
        elif size < 2000:
            d['1000-2000'] += 1
        elif size < 3000:
            d['2000-3000'] += 1
        elif size < 4000:
            d['3000-4000'] += 1
        elif size < 5000:
            d['4000-5000'] += 1
        elif size < 6000:
            d['5000-6000'] += 1
        elif size < 7000:
            d['6000-7000'] += 1
        elif size < 8000:
            d['7000-8000'] += 1
        elif size < 9000:
            d['8000-9000'] += 1
        elif size < 10000:
            d['9000-10000'] += 1
        else:
            d['>10000'] += 1

        if size > 2000:
            ls2.append([size, driv.dict_files_fullname[i['id']]])

        d['ls_files>5000'] = ls2
    return d

In [3]:
drivers = [
            ['Ci', '0AHj-PRcbOxSWUk9PVA'],
           ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
           ['Архив', '0AOP_oKFy4oGJUk9PVA'],
           ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
           ]
ls = []
for d in drivers:
    driv = GDrive(d[0], d[1])
    ls.append([d[0], ff1()])
ls

[['Ci',
  {'<1000': 7949,
   '1000-2000': 1,
   '2000-3000': 0,
   '3000-4000': 0,
   '4000-5000': 0,
   '5000-6000': 0,
   '6000-7000': 0,
   '7000-8000': 0,
   '8000-9000': 0,
   '9000-10000': 1,
   '>10000': 2,
   'ls_files>5000': [[30767.619193077087,
     'GoogleDriveTransfer/Ci/!MSSQL_Backup/InputIntS1.csv'],
    [48413.28955078125,
     'GoogleDriveTransfer/Ci/!MSSQL_Backup/MT_Input_History.bak'],
    [9358.09619140625,
     'GoogleDriveTransfer/Ci/!MSSQL_Backup/All_Int_2013.bak']]}],
 ['Managment 2',
  {'<1000': 21678,
   '1000-2000': 7,
   '2000-3000': 3,
   '3000-4000': 0,
   '4000-5000': 0,
   '5000-6000': 0,
   '6000-7000': 0,
   '7000-8000': 0,
   '8000-9000': 0,
   '9000-10000': 0,
   '>10000': 0,
   'ls_files>5000': [[2067.038091659546,
     'GoogleDriveTransfer/Managment 2/Р1/ЖК Питер 1353/Тех  часть/Аналог/ЭОМ.zip'],
    [2199.2690563201904,
     'GoogleDriveTransfer/Managment 2/ИНПИ/ЖК Мукомольный/Тех часть/2. РД 1 этап (03.07.20)/2. РД 1 этап (03.07.20)/I этап.zip'],

In [9]:
# [i[1]['ls_files>5000'] for i in ls]
ls1 = []
for i in ls:
    ls1.extend(i[1]['ls_files>5000'])
ls1

[[30767.619193077087, 'GoogleDriveTransfer/Ci/!MSSQL_Backup/InputIntS1.csv'],
 [48413.28955078125,
  'GoogleDriveTransfer/Ci/!MSSQL_Backup/MT_Input_History.bak'],
 [9358.09619140625, 'GoogleDriveTransfer/Ci/!MSSQL_Backup/All_Int_2013.bak'],
 [2067.038091659546,
  'GoogleDriveTransfer/Managment 2/Р1/ЖК Питер 1353/Тех  часть/Аналог/ЭОМ.zip'],
 [2199.2690563201904,
  'GoogleDriveTransfer/Managment 2/ИНПИ/ЖК Мукомольный/Тех часть/2. РД 1 этап (03.07.20)/2. РД 1 этап (03.07.20)/I этап.zip'],
 [2734.3365545272827,
  'GoogleDriveTransfer/Managment 2/Балтийский вагоноремонтный завод «Новотранс» (БВРЗ)/Тех часть/Проектная документация PDF_DWG.zip'],
 [4035.7817211151123,
  'GoogleDriveTransfer/Архив/idmitrieva/RENDERS/аттракцион_молот_судьбы.rar'],
 [4110.618448257446,
  'GoogleDriveTransfer/Архив/idmitrieva/RENDERS/Lobnya/facad/render/1.cxr'],
 [4110.618448257446,
  'GoogleDriveTransfer/Архив/idmitrieva/RENDERS/Lobnya/panorama/1.cxr'],
 [2640.7392950057983,
  'GoogleDriveTransfer/Архив/Граждан

In [29]:
ls[0][1]['ls_files>5000']

[[30767.619193077087, 'GoogleDriveTransfer/Ci/!MSSQL_Backup/InputIntS1.csv'],
 [48413.28955078125,
  'GoogleDriveTransfer/Ci/!MSSQL_Backup/MT_Input_History.bak'],
 [9358.09619140625, 'GoogleDriveTransfer/Ci/!MSSQL_Backup/All_Int_2013.bak']]

In [20]:
print(d)

{'<1000': 7949, '1000-2000': 1, '2000-3000': 0, '3000-4000': 0, '4000-5000': 0, '5000-6000': 0, '6000-7000': 0, '7000-8000': 0, '8000-9000': 0, '9000-10000': 1}


In [6]:
ls = []
for d in drivers:
    driv = GDrive(d[0], d[1])
    ls.append([d[0], len(driv.dict_files), len(driv.files_blobs)])
ls

[['Ci', 7953, 3998],
 ['Managment 2', 21688, 18406],
 ['Архив', 34662, 4652],
 ['Винный парк', 7761, 3222],
 ['ОСК Шереметьево', 1094, 1000]]

In [4]:
status.progress()

0.28258539784862924

In [8]:
status.progress(), status.resumable_progress / 1024 / 1024, status.total_size / 1024 / 1024

(1.0, 4246.50396156311, 4246.50396156311)

In [4]:
done = driv.download_files()

KeyboardInterrupt: 

In [6]:
blobs = driv.bucket.list_blobs(prefix='GoogleDriveTransfer/BackOffice')
files = [blob.name for blob in blobs]

In [6]:
gdf = driv.dict_files['1r5nHAOkTw5980x2shpleprf8hBJRHudm']

In [184]:
service, drive, bucket = get_client()

AttributeError: 'GoogleDrive' object has no attribute 'drives'

In [148]:
ls_drives = get_list_of_drives(service)

In [185]:
drive_name_id = ls_drives[23]
drive_name_id

['Мой Диск _', '0AHpbUawYyiEZUk9PVA']

In [186]:
gd1 = GDrive(drive, bucket, drive_name_id[0], drive_name_id[1])

In [109]:
list(gd1.dict_files.keys())[0]

'1SKwTfRGC4Td_4K5ClSIlOLRsiE59vLIwSegzgovQOGw'

In [110]:
def dict_num(dict, number):
    return list(dict.keys())[number]

In [187]:
file_id = dict_num(gd1.dict_files, 820)
file_id

'1u6CvHnjuyklQqQ4uuxkVNq9ehxv74MrL'

In [188]:
gdfile = gd1.dict_files[file_id]

In [189]:
namefile = gdfile['title']
fullname = gd1.dict_files_fullname[file_id]

In [190]:
fullname

'GoogleDriveTransfer/Мой Диск _/Юляша/IMG_20180628_193304.jpg'

In [175]:
file = drive.CreateFile({'id': file_id})

In [178]:
file.GetContentFile(namefile)

ApiRequestError: <HttpError 404 when requesting https://www.googleapis.com/drive/v2/files/1r82ugqPAlo2y3QwIBC1HvtS5TlG8iZcC?alt=json returned "File not found: 1r82ugqPAlo2y3QwIBC1HvtS5TlG8iZcC". Details: "File not found: 1r82ugqPAlo2y3QwIBC1HvtS5TlG8iZcC">

In [167]:
gdfile = gd1.dict_files[file_id]

In [168]:
mimeType = gdfile['mimeType']
mimeType

'image/jpeg'

In [169]:
namefile = gdfile['title']
fullname = gd1.dict_files_fullname[file_id]
# namefile = f"{namefile}.docx"
# fullname = f"{fullname}.docx"
namefile, fullname

('015.jpg',
 'GoogleDriveTransfer/BackOffice/Регламенты/Стандарт предприятия проектного института/Стандарт предприятия проектного института/Задание на проектирование/015.jpg')

In [193]:
blob = bucket.blob(fullname)

In [194]:
blob.exists()

False

In [174]:
s1 = gdfile.GetContentString()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [162]:
gdfile.GetContentFile(namefile, mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')

In [163]:
import os

# удалить файл
os.remove(namefile)

In [138]:
f = gdfile.GetContentFile(
    f"{namefile}, mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document'")

FileNotDownloadableError: No downloadLink/exportLinks for mimetype found in metadata

In [ ]:
f = gdfile.GetContentFile(f"'{namefile}', mimetype='{mimeType}'")

In [ ]:
def download_file(file_id):
    gdfile = gd1.dict_files[file_id]
    mimeType = gdfile['mimeType']
    namefile = gdfile['title']
    fullname = gd1.dict_files_fullname[file_id]

    if mimeType == 'application/vnd.google-apps.document':
        namefile = f"{namefile}.docx"
        fullname = f"{fullname}.docx"
        gdfile.GetContentFile(namefile,
                              mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')

    elif mimeType == 'application/vnd.google-apps.spreadsheet':
        namefile = f"{namefile}.xlsx"
        fullname = f"{fullname}.xlsx"
        gdfile.GetContentFile(namefile, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    elif mimeType == 'application/vnd.google-apps.presentation':
        namefile = f"{namefile}.pptx"
        fullname = f"{fullname}.pptx"
        gdfile.GetContentFile(namefile,
                              mimetype='application/vnd.openxmlformats-officedocument.presentationml.presentation')

    elif mimeType == 'application/vnd.google-apps.shortcut':
        namefile = f"{namefile}.url"
        fullname = f"{fullname}.url"
        gdfile.GetContentFile(namefile, mimetype='application/x-ms-shortcut')

    elif mimeType == 'application/vnd.google-apps.site':
        namefile = f"{namefile}.html"
        fullname = f"{fullname}.html"
        gdfile.GetContentFile(namefile, mimetype='text/html')

    blob = bucket.blob(fullname)
    blob.upload_from_filename(namefile)

    os.remove(namefile)


In [154]:
gdfile['title']

'Должностная инструкция инженера'

In [155]:
gdfile.GetContentFile(f"{gdfile['title']}.docx",
                      mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')

In [127]:
namefile = f"{gdfile['title']}.docx"
fullname = f"{gd1.dict_files_fullname[file_id]}.docx"
namefile, fullname

('Учет рабочего времени. Методика работы по Тайм трекингу CRM системы.docx',
 'GoogleDriveTransfer/BackOffice/СТП_2022/Учет рабочего времени. Методика работы по Тайм трекингу CRM системы.docx')

In [129]:

namefile = f"{gdfile['title']}.docx"
fullname = f"{gd1.dict_files_fullname[file_id]}.docx"

blob = bucket.blob(fullname)
blob.upload_from_filename(namefile)

In [128]:
blob.upload_from_filename(namefile)

NameError: name 'blob' is not defined

In [75]:
drive_name_id = ls_drives[1]
drive_name_id

['Ci', '0AHj-PRcbOxSWUk9PVA']

In [76]:
start_time = time.time()
ls_files = get_list_of_files_in_drive(service, drive_name_id)
end_time = time.time()

print('Время выполнения: ', end_time - start_time)

Время выполнения:  57.646464586257935


In [27]:
ls1 = []
for file in ls_files:
    ls1.append(get_full_path_to_file(service, drive_name_id, file['id']))

In [ ]:
response = service.files().list(
    q="mimeType!='application/vnd.google-apps.folder'",
    corpora='drive',
    driveId=drive_name_id[1],
    includeItemsFromAllDrives=True,
    supportsAllDrives=True,
    fields='nextPageToken, files(id, name, mimeType)').execute()

In [34]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

In [72]:
import time

In [29]:
gauth = GoogleAuth('savvy-concord-374520-bbbb0e3c73c3.json')
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)

KeyError: 'client_config_file'

In [30]:
gauth = GoogleAuth()

In [33]:
gauth.LoadCredentialsFile('savvy-concord-374520-bbbb0e3c73c3.json')

KeyError: '_module'

In [35]:
gauth = GoogleAuth()
scope = ["https://www.googleapis.com/auth/drive"]
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name('savvy-concord-374520-bbbb0e3c73c3.json', scope)
drive = GoogleDrive(gauth)

In [73]:
drive_name_id = ls_drives[1]
drive_name_id

['Ci', '0AHj-PRcbOxSWUk9PVA']

In [74]:


file_list = drive.ListFile(
    {
        'q': "mimeType!='application/vnd.google-apps.folder'",
        'corpora': 'drive',
        'driveId': drive_name_id[1],
        'includeItemsFromAllDrives': True,
        'supportsAllDrives': True,
    }
).GetList()




Время выполнения:  55.648189544677734


In [60]:
folder_list = drive.ListFile(
    {
        'q': "mimeType='application/vnd.google-apps.folder'",
        'corpora': 'drive',
        'driveId': drive_name_id[1],
        'includeItemsFromAllDrives': True,
        'supportsAllDrives': True,
    }
).GetList()

In [63]:
fd = {}
for folder in folder_list:
    fd[folder['id']] = [folder['title'], folder['parents'][0]['id']]

In [69]:
file_id = file_list[0]['parents'][0]['id']
file_id

'1ms7iki5PrYI6hPLIT1yqupxa-NmUv4T_'

In [64]:
drive_name_id[1]

'0AA9gFHffXEitUk9PVA'

In [70]:
def get_full_path(file_id):
    global fd
    tree = []
    while file_id != drive_name_id[1]:
        tree.append(fd[file_id][0])
        file_id = fd[file_id][1]
    tree.reverse()
    return '/'.join(tree)

In [71]:
get_full_path(file_id)

'Разработка/Смета/Сводный смета'

In [45]:
d = {}

for file in file_list:
    d[file['id']] = file['title']

In [ ]:
file.

In [40]:
print(file_list[0])

GoogleDriveFile({'kind': 'drive#file', 'userPermission': {'id': 'me', 'type': 'user', 'role': 'fileOrganizer', 'kind': 'drive#permission', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1Mrgtkgj1lYmBpmeyC6FSy3mUjHaAc5OT/permissions/me', 'etag': '"UnbmHQEv3L_pEM_YCAGzZyrISSg"'}, 'fileExtension': 'pdf', 'md5Checksum': 'a8bac911882f1cbba4cdbf12e6567346', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1Mrgtkgj1lYmBpmeyC6FSy3mUjHaAc5OT', 'lastModifyingUserName': '', 'editable': True, 'downloadUrl': 'https://www.googleapis.com/drive/v2/files/1Mrgtkgj1lYmBpmeyC6FSy3mUjHaAc5OT?alt=media&source=downloadUrl', 'mimeType': 'application/pdf', 'driveId': '0AA9gFHffXEitUk9PVA', 'parents': [{'selfLink': 'https://www.googleapis.com/drive/v2/files/1Mrgtkgj1lYmBpmeyC6FSy3mUjHaAc5OT/parents/1ms7iki5PrYI6hPLIT1yqupxa-NmUv4T_', 'id': '1ms7iki5PrYI6hPLIT1yqupxa-NmUv4T_', 'isRoot': False, 'kind': 'drive#parentReference', 'parentLink': 'https://www.googleapis.com/drive/v2/files/1ms7iki5PrYI6hPL

In [49]:
### Some basic helper functions ###
def get_children(root_folder_id):
    str = "\'" + root_folder_id + "\'" + " in parents and trashed=false"
    file_list = drive.ListFile(
        {
            'q': str,
            'corpora': 'drive',
            'driveId': drive_name_id[1],
            'includeItemsFromAllDrives': True,
            'supportsAllDrives': True,
        }).GetList()
    return file_list


def get_folder_id(root_folder_id, root_folder_title):
    file_list = get_children(root_folder_id)
    for file in file_list:
        if (file['title'] == root_folder_title):
            return file['id']


def add_children_to_tree(tree, file_list, parent_id):
    for file in file_list:
        tree.create_node(file['title'], file['id'], parent=parent_id)
        # For debugging
        # print('parent: %s, title: %s, id: %s' % (parent_id, file['title'], file['id']))


### Go down the tree until you reach a leaf ###
def populate_tree_recursively(tree, parent_id):
    children = get_children(parent_id)
    add_children_to_tree(tree, children, parent_id)
    if (len(children) > 0):
        for child in children:
            populate_tree_recursively(tree, child['id'])


In [54]:
from treelib import Node, Tree

In [52]:
root_folder_title = "КП"
root_folder_id = get_folder_id(drive_name_id[1], root_folder_title)

In [55]:
tree = Tree()

In [56]:
tree.create_node(root_folder_title, root_folder_id)

Node(tag=КП, identifier=1qO9iScUkSJVE0L2rs-dd4C1aNyJ9CzbI, data=None)

In [57]:
populate_tree_recursively(tree, root_folder_id)

In [58]:
tree.show()

КП
├── Волжский Композит - 3920-РАВ798-1-1 ТКП Ви-Гранд.pdf
├── Экополимер
│   ├── Буклет_Пром.предприятия.pdf
│   ├── Ситуационный план сооружений Model (1).pdf
│   └── ТКП № 100-20 ЗАО Читинские ключи_Анаэроб.pdf
└── Экополимер.rar



In [ ]:

### Create the tree and the top level node ###
def main():
    root_folder_title = "my-top-level-root-folder-name"
    root_folder_id = get_folder_id("root", root_folder_title)

    tree = Tree()
    tree.create_node(root_folder_title, root_folder_id)
    populate_tree_recursively(tree, root_folder_id)
    tree.show()

In [ ]:
# 1) Choose your starting point by inserting file name
folder_title = "КП"
folder_id = ''

In [195]:
drivers = [['BackOffice', '0AKtafUCJJsjKUk9PVA'],
 ['Grand_Engineering', '0APOKU4PVLZwMUk9PVA'],
 ['Human Resources', '0AJsRKr2Ba6TwUk9PVA'],
 ['IT-ОТДЕЛ', '0AI62IGV_POEcUk9PVA'],
 ['Managment 1', '0AJZoXjxIBMXRUk9PVA'],
 ['Titov Maksim', '0AAMdn20RTshcUk9PVA'],
 ['V-CLOUD24', '0AAUYrEjNjV5MUk9PVA'],
 ['Адвокатский кабинет', '0AHO8xAl6v2r5Uk9PVA'],
 ['Бухгалтер RUS', '0AKfDR0IMw1bPUk9PVA'],
 ['Бухгалтер UA', '0AGTAjdcjx6f-Uk9PVA'],
 ['ВДНХ_Колесо обозрения', '0ANw1UacIVifyUk9PVA'],
 ['ГипроМез', '0AKn-9Bj7NmNxUk9PVA'],
 ['ДОУ 360 Выпуск', '0AC2GUgCN9wcWUk9PVA'],
 ['ДОУ 360', '0ALpJRBJkrtVGUk9PVA'],
 ['Лио-Продукт', '0AHUPdJVPtsQaUk9PVA'],
 ['НКВЗ ремонт АвтоТехники', '0ADgaxv8NdpTkUk9PVA'],
 ['Объекты в работе', '0AOxaPQRooOk2Uk9PVA'],
 ['Остров мечты ППР', '0AImPD4jQdMXJUk9PVA'],
 ['Пекарня', '0AIWIhafOWgmHUk9PVA'],
 ['ПОДРЯДЧИКИ', '0ALnZgMLQFfrtUk9PVA'],
 ['СТО ГБО', '0AF2NtBbx_XvYUk9PVA'],
 ['ТЕХВАГОНМАШ', '0AAAyvFQMbstZUk9PVA'],
 ['Транспроект', '0ADKHIbPu575TUk9PVA'],
 ['ФосАгро', '0AD-4c3pLaBV5Uk9PVA'],
 ['ФРАКДЖЕТ-ТУЛЗ', '0ACjAWTbTqvESUk9PVA'],
 ['Холодный склад', '0ALT2Y1wkG7vCUk9PVA'],
 ['Чайка Лаб', '0APbGeMNQtN3MUk9PVA'],
 ['Читинские ключи', '0AA9gFHffXEitUk9PVA'],
 ['Юрист UA', '0AGg7JoJwa_8nUk9PVA']]
len(drivers)

29

In [199]:
drivers1 = [['Ci', '0AHj-PRcbOxSWUk9PVA'],
           ['Managment 2', '0AE3Z9Ox2YaurUk9PVA'],
           ['VG.COM', '0AElRg0lTw8iIUk9PVA'],
           ['Архив', '0AOP_oKFy4oGJUk9PVA'],
           ['Винный парк', '0ANhu4lfOrlBiUk9PVA'],
           ['Контент_VG', '0AN6chrg4ApnnUk9PVA'],
           ['Контент', '0ANf3LpamTrsTUk9PVA'],
           ['Лобня', '0AIQWy4IyyeaIUk9PVA'],
           ['Объединенная пивоварня', '0AELFNnmNadJqUk9PVA'],
           ['ОСК Шереметьево', '0ALRUo2DlF1YmUk9PVA'],
           ['Мой Диск _', '0AHpbUawYyiEZUk9PVA']
           ]
len(drivers1)

11

In [198]:
len(ls_drives)

40